# Weighted Interaction Graphs from Quantum Mutual Information

First we create a circuit to use.

In [2]:
import cirq

# Define a 3-qubit circuit
circuit = cirq.Circuit()
q0, q1, q2 = cirq.LineQubit.range(3)

# Apply Hadamard gates to each qubit
circuit.append([cirq.H(q0), cirq.H(q1), cirq.H(q2)])

# Apply CNOT gates to create a GHZ state
circuit.append([cirq.CNOT(q0, q1), cirq.CNOT(q1, q2)])

circuit

0: ───H───@───────
          │
1: ───H───X───@───
              │
2: ───H───────X───

In [3]:
rho = cirq.final_density_matrix(circuit)
rho

array([[0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j,
        0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j],
       [0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j,
        0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j],
       [0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j,
        0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j],
       [0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j,
        0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j],
       [0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j,
        0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j],
       [0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j,
        0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j],
       [0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j,
        0.12499998+0.j, 0.12499998+0.j, 0.12499998+0.j, 0.

Next, we define the function that computes the matrix of quantum mutaual information values. 

In [1]:
import numpy as np
from qiskit.quantum_info import partial_trace, entropy, mutual_information


def compute_mutual_information(rho):
    """
    Computes the quantum mutual information for every pair of qubits in the input density matrix.

    Args:
        rho (numpy.ndarray): a 2^n x 2^n density matrix representing the state of n qubits.

    Returns:
        numpy.ndarray: an n x n matrix of quantum mutual information values, where element i,j gives the mutual
        information between qubits i and j.
    """
    # Determine the number of qubits n from the shape of rho.
    n = int(np.log2(rho.shape[0]))

    # Create an empty n x n matrix to store the mutual information.
    mutual_information_matrix = np.zeros((n, n))

    # Loop over all pairs of qubits (i, j).
    for i in range(n):
        for j in range(i+1, n):

            # Define the indices of the qubits to trace out.
            trace_indices = [k for k in range(n) if k != i and k != j]

            # Trace out the other qubits to obtain the reduced density matrix of qubits i and j.
            rho_ij = partial_trace(rho, trace_indices)

            # Compute the von Neumann entropy of the reduced density matrix of qubit i.
            entropy_i = entropy(partial_trace(rho_ij, [1]))

            # Compute the von Neumann entropy of the reduced density matrix of qubit j.
            entropy_j = entropy(partial_trace(rho_ij, [0]))

            # Compute the mutual information between qubits i and j.
            mutual_information_matrix[i,j] = entropy_i + entropy_j - entropy(rho_ij)

    # Copy the upper triangular part of the matrix to the lower triangular part.
    mutual_information_matrix += mutual_information_matrix.T - np.diag(mutual_information_matrix.diagonal())

    return mutual_information_matrix


In [4]:
qmi = compute_mutual_information(rho)
qmi

array([[0.00000000e+00, 2.57973951e-07, 2.57973951e-07],
       [2.57973951e-07, 0.00000000e+00, 2.57973951e-07],
       [2.57973951e-07, 2.57973951e-07, 0.00000000e+00]])

In [5]:
import cirq

# Define a 3-qubit circuit
circuit2 = cirq.Circuit()
q0, q1, q2, q3 = cirq.LineQubit.range(4)

# Apply Hadamard gates to each qubit
circuit2.append([cirq.H(q0), cirq.H(q1), cirq.H(q2)])

# Apply CNOT gates to create a GHZ state
circuit2.append([cirq.CNOT(q0, q1), cirq.CNOT(q1, q2), cirq.CNOT(q2, q3)])

circuit2

0: ───H───@───────────
          │
1: ───H───X───@───────
              │
2: ───H───────X───@───
                  │
3: ───────────────X───